# 📊 Projeto de Teste Técnico — Cientista de Dados Júnior

* Este projeto tem como objetivo demonstrar habilidades analíticas, conhecimento em ciência de dados e capacidade de transformar dados brutos em insights acionáveis. Para isso, serão aplicadas técnicas de exploração, processamento e modelagem de dados, utilizando ferramentas e bibliotecas populares da área.


## 📌 Introdução

O desafio envolve a análise de um conjunto de dados fornecido, passando pelas seguintes etapas:

- **Entendimento dos Dados**: Compreensão da estrutura do dataset, as variáveis disponíveis.
- **Pré-processamento dos dados**: Identificação e remoção de outliers, tratamento de valores ausentes se tiver, engenharia de atributos.
- **Pré-processamento de texto (NLP)**: Aplicação de técnicas de **Processamento de Linguagem Natural (NLP)**, incluindo remoção de stopwords, stemming, lematização e vetorização de texto para melhorar a qualidade dos dados textuais.
- **Extração de outras fontes de dados**: Requisição feita na API do IBGE para a extração da informação sobre a região geográfica de cada estado.
- **Análise Exploratória de dados**: Entender como as variáveis estão distribuidas
- **Resolução das Perguntas Propostas**

1 - Quais são os estados e cidades com o maior número de transações?

2 - Como as transações são distribuídas ao longo do tempo (por mês/ano)?

3 - Houve um aumento ou diminuição no número de transações ao longo do tempo? Em quais estados isso é mais evidente?

4 - Gere um gráfico de série temporal mostrando a evolução das transações ao longo dos meses

5 - Proponha uma segmentação dos cartórios com base nos critérios que achar relevante.




### Dicionário de dados:

* id_cartorio: Variável numérica - regra: 1 cartório pode ter diversos tipos de transações
* texto: Variável string - Variável Texto - Possui informações sobre as transações, tipo de transacao, quem comprou e vendeu, tipo do imóvel e o tamanho
* Estado: Variável Categórica - Possui a sigla do Estado
* Cidade: Variável Categórica - Possui o nome das cidades
* data: Variável do tipo data - Possui o mês e o ano da transação

**Extraído via NLP**

* tamanho_imovel (metros quadrados): Variável numérica - poussui valores inteiros
* Tipo_transferência: Variável Categórica - Com valores sendo  "Usucapião Extrajudicial", "Recuperação de Crédito", "Compra e Venda"
* Tipo_imóvel: Variável Categórica - Com valores sendo "Casa", "Apartamento" e "Terreno Rural"

**Extra**

regiao: Variável categórica - Nome da região sendo "Norte, Nordeste, Centro Oeste, Sul, Sudeste"

In [ ]:
# Visualização e Manipulação de dados

import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats

#Requisicao HTTP
import requests

#Avaliacao da clusterizacao e modelo nao supervisionado
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [ ]:
# Manipulação de texto
import re
import spacy



#Processamento de texto
from fuzzywuzzy import fuzz, process
import string

# Codificacao de variáveis
from sklearn.preprocessing import LabelEncoder

# 📌 Entendimento dos Dados  

Nesta etapa, realizamos uma análise inicial do conjunto de dados para garantir que ele esteja adequado para as próximas fases do projeto. Esse processo inclui a verificação da estrutura dos dados, identificação de possíveis inconsistências e a definição de estratégias para tratá-las.  

Os principais aspectos analisados são:  

- **Extração dos Dados**: Importação e carregamento dos dados a partir da fonte original.
- **Verificação de Valores Ausentes**: Identificação de possíveis lacunas nos dados e definição de estratégias para lidar com elas.
- **Tipos das Variáveis**: Análise dos tipos de dados (numéricos, categóricos, texto, etc.) para garantir compatibilidade com os métodos de análise e modelagem.
- **Descrição Estatística**: Estatísticas descritivas, como média, mediana, desvio padrão e distribuição das variáveis.
- **Possíveis Problemas**: Por existir uma coluna de texto, precisamos avaliar quais técnicas seram úteis para capturar informações relevantes para análise


### Conceitos importantes.

**Compra e venda**: é o processo jurídico e financeiro que envolve a transferência de um imóvel de um vendedor para um comprador, com o pagamento de um preço acordado. Esse processo é regulamentado por leis específicar e envolve uma séria de etapas para garantir que a transação seja legal, segura e válida.


**Recuperação de crédito**: é o processo pelo qual uma instituição financeira, empresa ou credor busca recuperar valores devidos por clientes ou empresas que não cumpriram com suas obrigações financeiras, como o pgamento de dívidas ou empréstimos. Esse processo é uma parte essencial da gestão de crédito, garantindo que o credor congia minimizar perdas financeiras decorrentes de inadimplência,


**Usucapião Extrajudicial**: é um procedimento legal que permite a aquisição da propriedade de um bem imóvel por meio da posse contínua e ininterrupta por um determinado período, sem a necessidade de processo judicial. Esse processo ocorre diretamente no **cartório de registro de imóveis**, sendo uma alternativa mais rápida e menos burocrática em comparação com a usucapião judicial, que exige a intervenção do Poder Judiciário.


**Usucapião**: é a forma pela qual uma pessoa adquire a propriedade de um imóvel através da posse prolongada e sem oposição do proprietário anterior.



In [300]:
# Extraindo os dados
dados = pd.read_csv('dataset_registros_imobiliarios.csv')

In [301]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10485 entries, 0 to 10484
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_cartorio  10485 non-null  int64 
 1   texto        10485 non-null  object
 2   estado       10485 non-null  object
 3   cidade       10485 non-null  object
 4   data         10485 non-null  object
dtypes: int64(1), object(4)
memory usage: 409.7+ KB


# Pré-processamento

* A coluna de data se encontra no tipo 'object', será feito uma transformação para que seja transmita sua informação de maneira mais concreta.

In [302]:
dados.head()

,id_cartorio,texto,estado,cidade,data
0,193,Processo de Usucapião Extrajudicial: Apartamen...,PI,Picos,08/2023
1,276,Processo de Usucapião Extrajudicial: Terreno R...,MA,São Luís,11/2023
2,70,Processo de Usucapião Extrajudicial: Terreno R...,SC,Blumenau,02/2024
3,277,Casa (291m²) transferido de Ana Beatriz da Roc...,CE,Fortaleza,04/2024
4,166,Processo de Compra e Venda: Casa de 153m² pass...,RJ,Niterói,10/2024


In [303]:
dados['data'] = pd.to_datetime(dados['data'], format = '%m/%Y')
dados.dtypes

id_cartorio             int64
texto                  object
estado                 object
cidade                 object
data           datetime64[ns]
dtype: object

A **engenharia de atributos** é um processo fundamental no ciclo de vida de um projeto de ciência de dados. Consiste na criação, modificação ou seleção de variáveis (atributos) que serão utilizadas nos modelos de machine learning, com o objetivo de melhorar a performance e a capacidade preditiva dos modelos.

Esse processo pode ser dividido em várias etapas, dependendo dos dados e do problema que estamos tentando resolver. Algumas das principais atividades envolvem:

 1. **Criação de Atributos**
A criação de novos atributos pode ser feita a partir de variáveis já existentes. Isso pode incluir:
- Combinação de várias colunas para gerar uma nova variável (por exemplo, somar a idade e a experiência para criar uma variável de "experiência total").
- Conversão de variáveis categóricas em variáveis numéricas (como o uso de one-hot encoding ou label encoding).
- Extração de informações de texto, como mostrado anteriormente, para capturar dados importantes que podem não ser explícitos na estrutura original dos dados.

2. **Transformação de Atributos**
Em alguns casos, as variáveis precisam ser transformadas para tornar os modelos mais eficientes:
- **Escalonamento de variáveis** (normalização ou padronização) para garantir que as variáveis numéricas estejam na mesma escala, evitando que variáveis com maior magnitude dominem a análise.
- **Log-transform** para lidar com distribuições assimétricas ou com grandes variações entre os dados.


No nosso caso, criamos a variável mês e ano a partir da coluna de data.

In [304]:
# Agora com a coluna transformada, iremos extrair o mes e o ano
dados['ano'] = dados['data'].dt.year
dados['mes'] = dados['data'].dt.month
dados.head(2)

,id_cartorio,texto,estado,cidade,data,ano,mes
0,193,Processo de Usucapião Extrajudicial: Apartamen...,PI,Picos,2023-08-01,2023,8
1,276,Processo de Usucapião Extrajudicial: Terreno R...,MA,São Luís,2023-11-01,2023,11


🔍 Extração de Informações de Texto

A coluna de texto contém frases longas, que podem incluir informações valiosas para futuras análises. No entanto, esses dados não estruturados precisam ser processados para que possamos extrair insights úteis. Alguns dos elementos que podem ser extraídos incluem:

- **Tipo de Transferência**: Identificação do tipo de transação imobiliária realizada.
- **Tipo do Imóvel**: Classificação do imóvel, como casa, apartamento, terreno, entre outros.
- **Tamanho do Imóvel**: Extração de informações sobre área construída ou área total.

Para tornar essas informações utilizáveis em nossas análises, é essencial realizar um **pré-processamento de texto**. Esse processo envolve a limpeza e estruturação dos dados textuais, o que pode incluir:

✅ **Remoção de caracteres especiais e stopwords** para evitar ruídos na análise.

✅ **Tokenização** para segmentar as frases em palavras individuais.

✅ **Identificação de padrões e extração de palavras-chave** para capturar as informações relevantes.

Esse tratamento garante que os dados textuais sejam transformados em variáveis estruturadas, facilitando análises futuras e contribuindo para uma modelagem mais eficiente.

Esse processo é conhecido como processamento de linguagem natural, será melhor definido mais a frente


In [305]:
# Como está estruturado a coluna de texto
for i in range(-5, 0):
    print(dados['texto'].iloc[i])

Processo de Recuperação de Crédito: Casa de 277m² passou de Ferreira (90.163.257/0001-57) para Raul Monteiro (176.384.259-28).
Apartamento de 204m² transferido de Ryan Novaes (146.985.027-30) para Mirella Castro (470.836.259-56) via Usucapião Extrajudicial.
Terreno Rural de 43156m² transferido de Mendonça (73.548.169/0001-54) para Araújo (85.641.397/0001-53) via Recuperação de Crédito.
Apartamento de 204m² transferido de Cirino e Filhos (76.025.983/0001-82) para Ribeiro (70.246.983/0001-26) via Compra e Venda.
Casa de 199m², Usucapião Extrajudicial realizado entre Rafaela da Luz (301.879.425-79) e Moraes Andrade - ME (65.984.172/0001-92).


# Pré-processamento de texto (NLP)

**Processamento de Linguagem Natural (NLP)** é uma subárea da Inteligência Artificial (IA) que envolve a interação entre computadores e a linguagem humana. O principal objetivo do NLP é permitir que os computadores compreendam, interpretem, analisem e gerem linguagem natural de maneira útil e significativa.

### Fuzzy Matching 

**Fuzzy Matching** é uma técnica que permite encontrar correspondências aproximadas entre strings (sequências de caracteres), ou seja, não é necessário que as palavras ou frases sejam idênticas. Em vez de exigir uma correspondência exata, o Fuzzy Matching encontra strings que são **semelhantes**, mesmo quando há erros de digitação, variações de escrita ou pequenas diferenças.

Por exemplo, ao tentar encontrar a palavra "apartamento", o Fuzzy Matching pode identificar palavras semelhantes, como "apartamanto" ou "apartemanto", mesmo que haja erros de digitação.

O Fuzzy Matching é útil em várias situações, como:

- **Correção de erros de digitação** em buscas de texto.
- **Identificação de palavras semelhantes**, mesmo que tenham variações ortográficas.
- **Busca aproximada** em bases de dados ou sistemas de recomendação.

Como o Fuzzy Matching Funciona

O Fuzzy Matching funciona calculando a **semelhança** entre duas strings usando algoritmos específicos, como:

- **Distância de Levenshtein**: Mede o número mínimo de edições (inserções, exclusões ou substituições) necessárias para transformar uma string em outra. Quanto menor a distância, maior a similaridade entre as palavras.
- **Algoritmo de Jaccard**: Mede a similaridade entre conjuntos de palavras ou caracteres.
- **Algoritmo de Jaro-Winkler**: É uma variação da distância de Jaro que penaliza mais as diferenças no início das palavras, que são mais significativas.

Que no nosso caso, estamos utilizando o **scorer** que é uma variação do cálculo de distância de Levenshtein. O que ele faz é comparar a palavra do texto com as opções da lista (no caso, os tipos de imoveis, tipo de transferencia) e calcular a distância de Levenshtein. Ele considera o alinhamento da palavra no texto com as palavras da lista, permitindo uma correspondÇencia mais flexível, até mesmo quando a palavra no texto é apenas uma parte da palavra da lista

Diferença entre Fuzzy Matching e Outras Bibliotecas de NLP

O Fuzzy Matching é diferente de outras bibliotecas de NLP porque seu foco não está na **análise semântica** ou na **compreensão profunda** do texto, mas sim em **comparar e medir a similaridade entre palavras ou frases**.

Outro ponto interessante do Fuzzy, é que ele se mostrou mais rápido em alguns processos do que modelos de nlp.

Enquanto o Fuzzy Matching se concentra em encontrar correspondências aproximadas de strings, as bibliotecas de NLP muitas vezes analisam o significado das palavras e as relações contextuais entre elas, utilizando modelos baseados em aprendizado profundo (deep learning). No nosso caso, Fuzzy apresentou ser bastante eficiente para o problema atual


Foi escolhido o modelo "pt_core_news_sm" porque ele foi treinado para entender a gramática, sintaxe e semântica do português.

In [306]:
# Carregar modelo de processamento de linguagem natural em português
nlp = spacy.load("pt_core_news_sm")

In [307]:
# Limpa e pré-processa o texto removendo pontuação, caracteres especiais e stopwords.

def preprocessar_texto(texto):

    # Remover caracteres especiais e pontuação
    texto = texto.translate(str.maketrans("", "", string.punctuation))
    texto = re.sub(r'[@#\$%\&\*\(\)\+\=\[\]\{\};:<>/\\,.\?¡!]', '', texto)

    # Criar o objeto doc usando spaCy
    doc = nlp(texto)

    # Remover stopwords e manter apenas tokens relevantes
    tokens_processados = [token.text for token in doc if not token.is_stop]

    # Reconstruir o texto processado
    texto_processado = " ".join(tokens_processados)
    return texto_processado

def extrair_tamanho(texto):
    # Regex para capturar o tamanho do imóvel (em m², m, ou número apenas)
    match = re.search(r'(\d+)(?:[ ]?m²?|m)?', texto)

    if match:
        return int(match.group(1))

    return None

No contexto de análise de texto, como no caso da extração de tipos de imóvel e transações, o uso de Fuzzy Matching pode ser mais vantajoso em comparação com abordagens baseadas em modelos de linguagem natural (NLP) tradicionais, como o spaCy, Foi escolhido o Fuzzy por alguns motivos:

- *1 Variações de Texto: Muitas vezes, os textos podem conter erros de digitação, abreviações ou variações na forma como os termos são escritos. O Fuzzy Matching permite que se identifique correspondências aproximadas entre palavras ou frases, mesmo que estas não coincidam exatamente. Isso é útil para lidar com erros de digitação ou expressões que não sejam padronizadas.

- *2 Simplicidade: O uso de Fuzzy Matching é mais simples e direto quando os tipos de dados a serem extraídos são conhecidos e limitados, como listas de tipos de imóvel ou transações. Ao usar Fuzzy Matching, a comparação é feita diretamente entre a entrada do usuário (texto) e os possíveis tipos, sem a necessidade de um modelo complexo de NLP.

- 3* Performance: O Fuzzy Matching, especialmente com algoritmos como o Levenshtein Distance (usado pelo fuzz.partial_ratio), oferece uma solução computacionalmente mais eficiente para tarefas de correspondência aproximada entre texto curto (como os nomes de tipos de imóveis ou transações) em comparação com o uso de modelos pesados de NLP.

O modelo carregado foi testado em algumas atividades, como a extração do tipo de imóvel, e apresentou um custo computacional significativamente maior, resultando em uma execução mais lenta da função. Para tarefas simples de correspondência exata, como a que estamos realizando, o uso do spaCy pode ser mais custoso do que realmente necessário. Por esse motivo, o spaCy foi utilizado com parcimônia durante o processo, enquanto o Fuzzy Matching teve maior ênfase, pois é mais rápido e eficiente para essas tarefas específicas.

In [308]:

def extrair_tipo_imovel(texto):
    tipos_imovel = ["casa", "apartamento", "terreno rural"]

    # Tokeniza o texto para trabalhar palavra por palavra
    palavras = texto.split()

    tipo_imovel_encontrado = None
    maior_score = 0

    for palavra in palavras:
        # Faz o Fuzzy Matching para comparar cada palavra
        melhor_match, score = process.extractOne(palavra, tipos_imovel, scorer=fuzz.partial_ratio)

        if score > maior_score:
            maior_score = score
            tipo_imovel_encontrado = melhor_match

    return tipo_imovel_encontrado if maior_score > 80 else None

def extrair_tipo_transacao(texto):
    # Definir os tipos de transações comuns
    tipos_transacao = ["Compra e Venda", "Usucapião Extrajudicial", "Recuperação de Crédito"]

    # Criar uma lista de possíveis correspondências
    tipo_transacao_encontrado = None
    for transacao in tipos_transacao:

        # Fuzzy matching para lidar com variações
        match = process.extractOne(transacao, texto.split(), scorer=fuzz.partial_ratio)
        if match and match[1] > 80:  # Define a confiança de 80% para considerar a correspondência
            tipo_transacao_encontrado = transacao
            break

    return tipo_transacao_encontrado

In [309]:
# Limparemos o texto primeiro para evitar problemáticas a frente
dados['texto'] = dados['texto'].apply(preprocessar_texto)
dados['tamanho_imovel'] = dados['texto'].apply(extrair_tamanho)

In [310]:
dados['tipo_transferencia'] = dados['texto'].apply(extrair_tipo_transacao)

In [311]:
dados['tipo_imovel'] = dados['texto'].apply(extrair_tipo_imovel)

A função extrair_tipo_transacao ou extrair_tipo_imovel retorna None quando não encontra uma correspondência com os tipos de transação definidos na lista tipos_transacao. Isso ocorre porque, no bloco de código onde realizamos o fuzzy matching com a função process.extractOne, comparamos a transação do texto com as transações pré-definidas. Caso o score de confiança da correspondência seja inferior a 80, a função não considera a transação como válida e retorna None.

O retorno de None tem como objetivo facilitar a identificação dos casos em que o tipo de transação não foi adequadamente identificado ou registrado. Isso permite que, ao utilizarmos dados.isnull().sum(), possamos verificar rapidamente quais registros possuem valores ausentes (nulos) e precisam ser corrigidos ou ajustados, seja por erro de digitação ou por outros fatores que impediram a identificação correta do tipo de transação. Essa abordagem facilita a limpeza e a organização dos dados antes de realizar análises mais aprofundadas.

In [312]:
dados.isnull().sum()

id_cartorio           0
texto                 0
estado                0
cidade                0
data                  0
ano                   0
mes                   0
tamanho_imovel        0
tipo_transferencia    0
tipo_imovel           2
dtype: int64

In [313]:
# Verificando as linhas que não foram atribuidas valor do tipo_imovel devidamente
dados[dados['tipo_imovel'].isnull()]

,id_cartorio,texto,estado,cidade,data,ano,mes,tamanho_imovel,tipo_transferencia,tipo_imovel
3108,54,Csa 194 m² transferido José Miguel Paz 8910642...,RJ,Niterói,2023-03-01,2023,3,194,Usucapião Extrajudicial,None
9909,75,Caa 237 m² transferido Sousa 45681390000140 Sr...,SE,Nossa Senhora do Socorro,2024-01-01,2024,1,237,Usucapião Extrajudicial,None


Como dois textos apresentaram o nome da casa digitado incorretamente após o processamento, foi decidida a utilização de um replace para corrigir esses erros na coluna. Após essa correção, a função para extrair os tipos de imóveis foi executada novamente para garantir a precisão dos dados.

In [314]:
# Condição para selecionar cartórios 54 ou 75
indice_alteracao = dados['id_cartorio'].isin([54, 75])

# Substituindo 'Csa' e 'Caa' por 'Casa' na coluna 'texto' onde a condição é verdadeira
dados.loc[indice_alteracao , 'texto'] = dados.loc[indice_alteracao , 'texto'].replace({'Csa': 'Casa', 'Caa': 'Casa'}, regex=True)

In [315]:
dados[dados['tipo_imovel'].isnull()]

,id_cartorio,texto,estado,cidade,data,ano,mes,tamanho_imovel,tipo_transferencia,tipo_imovel
3108,54,Casa 194 m² transferido José Miguel Paz 891064...,RJ,Niterói,2023-03-01,2023,3,194,Usucapião Extrajudicial,None
9909,75,Casa 237 m² transferido Sousa 45681390000140 S...,SE,Nossa Senhora do Socorro,2024-01-01,2024,1,237,Usucapião Extrajudicial,None


In [316]:
# Realizando o processo novamente
dados['tipo_imovel'] = dados['texto'].apply(extrair_tipo_imovel)

In [317]:
# Verificando se ouve algum caso onde algum valor não foi colocado
dados.isnull().sum()

id_cartorio           0
texto                 0
estado                0
cidade                0
data                  0
ano                   0
mes                   0
tamanho_imovel        0
tipo_transferencia    0
tipo_imovel           0
dtype: int64

### Análise de Inconsistências no Conjunto de Dados

Durante a análise, identificamos registros com tamanhos de imóveis irreais, como o caso de imóveis com apenas 2 metros quadrados. Esses registros foram considerados inconsistentes e, portanto, excluídos. Especificamente, foram encontrados 2 casos com essa característica:
ID Cartório 51 e 7.
Esses registros foram tratados para garantir a qualidade dos dados e a precisão das análises subsequentes.

In [318]:
tamanho = dados['tamanho_imovel'].min()
dados[dados['tamanho_imovel'] == tamanho]

,id_cartorio,texto,estado,cidade,data,ano,mes,tamanho_imovel,tipo_transferencia,tipo_imovel
2246,51,Casa 2 m² transferido Sra Maria Júlia Albuquer...,PI,Parnaíba,2024-06-01,2024,6,2,Compra e Venda,casa
2750,7,Casa 2 m² transferido Rios 92641875000190 Anth...,AP,Macapá,2024-02-01,2024,2,2,Recuperação de Crédito,casa


In [319]:
indices_para_remover = dados[dados['id_cartorio'].isin([51, 7])].index
dados = dados.drop(indices_para_remover)

Análise de Transações por Ano e Identificação de Possíveis Outliers
Durante a análise dos registros de transações agrupados por ano, observamos uma anomalia no ano de 2025. Este ano apresenta um número de transações significativamente baixo, sugerindo que os dados podem estar incompletos. O último registro disponível é de fevereiro, um mês que ainda não se completou, o que pode indicar que o conjunto de dados está incompleto até o momento, caracterizando-se como um possível outlier.

Além disso, a análise da série temporal revelou que o mês de fevereiro de 2023 também apresenta um valor abaixo da média, com apenas 224 registros. Esse dado também deve ser tratado com cautela, pois, ao analisar séries temporais, a presença de meses com número reduzido de registros pode distorcer as conclusões.

É importante ressaltar que, ao observar o crescimento dos meses anteriores, pode-se observar uma tendência de aumento nas transações. Contudo, ao chegarmos a 2025, especialmente em fevereiro, é possível que uma aparente queda seja observada, devido à possível falta de dados completos para esse ano. Essa redução aparente pode ser apenas um reflexo da falta de registros suficientes para representar fielmente o mês, o que é esperado, já que o mês de fevereiro ainda não terminou.

In [320]:
ano_agrupados = dados.groupby('ano').size().reset_index(name = 'contagem')
ano_agrupados = ano_agrupados.sort_values(by = 'contagem', ascending= False)
ano_agrupados

,ano,contagem
1,2024,5214
0,2023,4567
2,2025,636


In [321]:
data_agrupada = dados.groupby('data').size().reset_index(name = 'contagem')
data_agrupada = data_agrupada.sort_values(by = 'data')
data_agrupada

,data,contagem
0,2023-02-01,224
1,2023-03-01,412
2,2023-04-01,448
3,2023-05-01,466
4,2023-06-01,406
5,2023-07-01,461
6,2023-08-01,480
7,2023-09-01,406
8,2023-10-01,439
9,2023-11-01,419


In [322]:
print("Média da quantidade de transacoes por ano:",data_agrupada['contagem'].mean())

Média da quantidade de transacoes por ano: 416.68


# Extração de dados de outras fontes

Para enriquecer o conjunto de dados, utilizamos informações sobre os estados brasileiros e suas respectivas regiões geográficas. Esses dados foram obtidos diretamente da **API pública do IBGE**, garantindo que as informações estejam sempre atualizadas e estruturadas.

A obtenção dos dados foi realizada através de uma **requisição HTTP**, resultando em um arquivo **JSON** contendo a sigla do estado e sua respectiva região geográfica. Para facilitar a manipulação e integração com nosso conjunto de dados, os dados foram convertidos para um **DataFrame do pandas**.

Essa abordagem permitiu uma integração dinâmica e confiável, garantindo que qualquer atualização realizada pelo IBGE seja refletida automaticamente em nosso sistema.

In [323]:
url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados"
response = requests.get(url)
estados_dados = response.json()
dados_regioes = pd.DataFrame([(estado['sigla'], estado['regiao']['nome']) for estado in estados_dados], columns = ['estado', 'regiao'])

In [324]:
# Merge junta os dois dataframes pela coluna sigla
dados = dados.merge(dados_regioes, on = 'estado', how = 'left')
dados.head(3)

,id_cartorio,texto,estado,cidade,data,ano,mes,tamanho_imovel,tipo_transferencia,tipo_imovel,regiao
0,193,Processo Usucapião Extrajudicial Apartamento 1...,PI,Picos,2023-08-01,2023,8,121,Usucapião Extrajudicial,apartamento,Nordeste
1,276,Processo Usucapião Extrajudicial Terreno Rural...,MA,São Luís,2023-11-01,2023,11,35049,Usucapião Extrajudicial,terreno rural,Nordeste
2,70,Processo Usucapião Extrajudicial Terreno Rural...,SC,Blumenau,2024-02-01,2024,2,28034,Usucapião Extrajudicial,terreno rural,Sul


# Análise Exploratória



Observa-se que o imóvel do tipo "apartamento" apresenta um valor de tamanho maior em comparação aos outros tipos de imóvel, embora a diferença não seja muito expressiva.

In [325]:
tipo_imovel = dados.groupby('tipo_imovel').size().reset_index(name='contagem')

fig1 = px.bar(tipo_imovel,
              x='tipo_imovel',
              y='contagem',
              title='Distribuição do Tipo de Imóvel',
              labels={'contagem': 'Número de Registros', 'tipo_imovel': 'Imóvel'},
              color_discrete_sequence=['royalblue'])


# Ajuste para melhorar a exibição das barras
fig1.update_traces(
    texttemplate='%{y}',
    textposition='inside',
    marker=dict(line=dict(color='black', width=1))
)

fig1.update_layout(
    xaxis_tickangle=0,
    plot_bgcolor='white',
    showlegend=False,
    bargap=0.15
)

fig1.show()

O processo de compra e venda apresenta um valor superior em relação aos outros processos analisados. No entanto, a diferença não é tão expressiva, indicando que, embora haja uma variação, os outros tipos de transação, como usucapião extrajudicial e recuperação de crédito, possuem valores próximos. Isso pode sugerir que as transações de compra e venda são frequentes, mas não são, de maneira significativa, mais representativas ou impactantes do que os demais processos, pelo menos dentro do contexto dos dados analisados. A ausência de uma grande disparidade entre os valores sugere um equilíbrio nas transações realizadas em comparação com os outros tipos de processo.

In [326]:
# Agrupar os dados pelo tipo de transferência
tipo_transferencia = dados.groupby('tipo_transferencia').size().reset_index(name='contagem')

fig1 = px.bar(tipo_transferencia,
              x='tipo_transferencia',
              y='contagem',
              title='Distribuição do Tipo de Transferência',
              labels={'contagem': 'Número de Registros', 'tipo_transferencia': 'Tipo de Transferência'})

# Ajuste para melhorar a exibição das barras
fig1.update_traces(
    texttemplate='%{y}',
    textposition='inside',
    marker=dict(color='royalblue', line=dict(color='black', width=1))
)

fig1.update_layout(
    xaxis_tickangle=0,
    plot_bgcolor='white',
    showlegend=False,
    bargap=0.15
)

fig1.show()

A suavização logarítmica foi aplicada devido à grande disparidade nos valores dos tamanhos dos imóveis, especialmente no caso dos terrenos rurais. Esses terrenos possuem valores significativamente mais altos, o que é esperado, já que podem representar áreas como fazendas, sítios ou terrenos para cultivo. Para lidar com essa diferença acentuada, utilizamos a mediana como medida de tendência central, pois ela é menos sensível a valores discrepantes, o que garante uma comparação mais robusta.

A análise sugere que os tamanhos dos terrenos rurais, de fato, apresentam valores muito elevados em relação aos outros tipos de imóveis. Esse fato corrobora a hipótese de que qualquer comparação envolvendo a coluna de tamanhos de imóveis deve ser feita com cautela, especialmente quando envolve terrenos rurais, devido à sua magnitude. Portanto, qualquer análise comparativa envolvendo esses valores precisa ser moderada, considerando a natureza excepcional dos dados relacionados aos terrenos rurais.

In [327]:
mediana_tamanho_por_tipo = dados.groupby('tipo_imovel')['tamanho_imovel'].mean().reset_index(name='mediana_tamanho')

# Aplicando transformação logarítmica para suavizar grandes discrepâncias
mediana_tamanho_por_tipo['mediana_log'] = np.log1p(mediana_tamanho_por_tipo['mediana_tamanho'])

fig = px.bar(mediana_tamanho_por_tipo,
             x='tipo_imovel',
             y='mediana_log',
             title='Mediana do Tamanho do Imóvel por Tipo de Imóvel (Escala Logarítmica)',
             labels={'mediana_log': 'Mediana do Tamanho do Imóvel (Log)', 'tipo_imovel': 'Tipo de Imóvel'},
             color='mediana_log',
             color_continuous_scale='blues')


# Ajuste para melhorar a exibição das barras
fig.update_traces(
    texttemplate='%{y:.2f}',
    textposition='inside',
    marker=dict(color='royalblue', line=dict(color='black', width=1))
)

fig.update_layout(
    xaxis_tickangle=360,
    plot_bgcolor='white',
    showlegend=False,
    bargap=0.15
)

fig.show()

# Solução das Perguntas


# 1- Quais são os estados e cidades com o maior número de transações?

* Para responder essa pergunta foi criado um gráfico de barras para cada grupo. Estados e cidades.

In [328]:
#Agrupando por estado e cidades
estados_agrupados = dados.groupby('estado').size().reset_index(name='quantidade_registros')
cidades_agrupados = dados.groupby('cidade').size().reset_index(name='quantidade_registros')

#Ordenando por quantidade de registros
estados_agrupados = estados_agrupados.sort_values(by = 'quantidade_registros', ascending= False)
cidades_agrupados = cidades_agrupados.sort_values(by = 'quantidade_registros', ascending= False)

top_10_estados = estados_agrupados.head(10)
top_10_cidade = cidades_agrupados.head(10)

In [329]:
# Gráfico para Estados
fig1 = px.bar(top_10_estados,
              x='estado',
              y='quantidade_registros',
              title='Número de Registros por Estado',
              labels={'quantidade_registros': 'Número de Registros', 'estado': 'Estado'},
              color='quantidade_registros',
              color_continuous_scale='blues')

fig1.update_traces(texttemplate='%{y}', textposition='inside', marker=dict(line=dict(color='black', width=1)))
fig1.update_layout(showlegend=False)  # Remover a legenda
fig1.show()

# Gráfico para Cidades
fig2 = px.bar(top_10_cidade,
              x='cidade',
              y='quantidade_registros',
              title='Número de Registros por Cidade',
              labels={'quantidade_registros': 'Número de Registros', 'cidade': 'Cidade'},
              color='quantidade_registros',
              color_continuous_scale='greens')

fig2.update_traces(texttemplate='%{y}', textposition='inside', marker=dict(line=dict(color='black', width=1)))
fig2.update_layout(showlegend=False)  # Remover a legenda
fig2.show()

**Notas**

O gráfico superior mostra os 10 estados com o maior número de transações, enquanto o gráfico inferior exibe as 10 cidades com o maior volume de registros. Ambos os gráficos estão organizados de forma decrescente, proporcionando uma visualização clara das regiões com maior concentração de transações, facilitando a análise das áreas de maior atividade.


# 2 - Como as transações são distribuídas ao longo do tempo (por mês/ano)?

Dois gráficos serão gerados para esta análise. Conforme discutido anteriormente, há indícios de que o primeiro e o último mês podem ser considerados outliers, potencialmente distorcendo a análise das transações ao longo do tempo. A abordagem escolhida consiste em plotar ambos os gráficos — um que inclui todos os meses e outro que exclui os meses extremos — para comparar as diferenças entre eles. Dessa forma, será possível avaliar o impacto desses valores e tirar conclusões sobre o comportamento dos dados.

In [330]:
transacoes_por_mes = dados.groupby('data').size().reset_index(name='quantidade_transacoes')

# Gráfico 1: Incluindo todos os meses
media_transacoes_completos = transacoes_por_mes['quantidade_transacoes'].mean()

fig1 = px.line(transacoes_por_mes, x='data', y='quantidade_transacoes',
               title='Distribuição de Transações ao Longo do Tempo (Todos os Meses)',
               labels={'quantidade_transacoes': 'Número de Transações', 'data': 'Mês/Ano'},
               markers=True)

# Adicionando a linha de média no primeiro gráfico
fig1.add_hline(y=media_transacoes_completos, line_dash="dot", line_color="red",
               annotation_text=f"Média: {media_transacoes_completos:.2f}",
               annotation_position="top left")

# Formatando o eixo X no primeiro gráfico
fig1.update_layout(xaxis=dict(tickformat="%m/%Y"))

# Gráfico 2: Excluindo o primeiro e o último mês
transacoes_por_mes_filtrado = transacoes_por_mes.sort_values(by='data').iloc[1:-1]
media_transacoes_filtrado = transacoes_por_mes_filtrado['quantidade_transacoes'].mean()



fig2 = px.line(transacoes_por_mes_filtrado, x='data', y='quantidade_transacoes',
               title='Distribuição de Transações ao Longo do Tempo (Excluindo 1º e Último Mês)',
               labels={'quantidade_transacoes': 'Número de Transações', 'data': 'Mês/Ano'},
               markers=True)

# Adicionando a linha de média no segundo gráfico
fig2.add_hline(y=media_transacoes_filtrado, line_dash="dot", line_color="red",
               annotation_text=f"Média: {media_transacoes_filtrado:.2f}",
               annotation_position="top left")

# Formatando o eixo X no segundo gráfico
fig2.update_layout(xaxis=dict(tickformat="%m/%Y"))

fig1.show()
fig2.show()

Nos dois gráficos apresentados inicialmente, é possível observar um crescimento nas transações, mas com diferenças notáveis entre eles. No primeiro gráfico, que inclui todos os meses, é evidente uma tendência acentuada de queda no final, o que resulta em valores significativamente mais baixos em comparação ao segundo gráfico. Este, por sua vez, exclui o primeiro e o último mês, e apresenta uma trajetória de crescimento mais consistente, especialmente a partir de novembro de 2024. O primeiro gráfico se manteve acima da média por um período mais longo, mas a queda abrupta no final foi um fator de desvio importante. Já o segundo gráfico, sem os meses extremos, apresentou um comportamento mais equilibrado, alternando altos e baixos, sem um claro período de estabilidade.

Um possível fator que pode explicar essa diferença nos comportamentos de transações ao longo do tempo está relacionado à falta de estratégias adequadas no setor imobiliário. A ausência de ações eficazes por parte dos profissionais do mercado imobiliário para se adaptarem às mudanças econômicas e comportamentais pode impactar diretamente as vendas. Além disso, a falta de uma interpretação correta do cenário do mercado e a falta de adaptação frente à atual situação econômica são fatores que podem contribuir para a diminuição das transações, como evidenciado nos gráficos.

Fonte: [Rankim - Diminuição de vendas de imóveis: descubra agora o porquê](https://rankim.com.br/blog/diminuicao-de-venda-de-imoveis-descubra-agora-o-porque)

# 3 - Houve um aumento ou diminuição no número de transações ao longo do tempo? Em quais estados isso é mais evidente?

* No gráfico é possível notar que as transações ao longo do tempo mantiveram-se na casa das 400 transações, para ser mais exato possui uma média de 419.40, logo ao ínicio do ano de 2025, teve uma queda 
brusca.

* Para ver quais estados isso é mais evidente iremos calcular a variação percentual. Iremos calcular o crescimento relativo ou diminuição relativa de um valor em comparação com um período anterior.

Iremos usar a formula:

$$

\text{Variação Percentual} = \left( \frac{\text{Valor de 2024} - \text{Valor de 2023}}{\text{Valor de 2023}} \right) \times 100

$$



* Foi decido usar os anos 2023 e 2024 porque possuem os meses completos em comparação a 2025, que se só possui janeiro e fevereiro. Para os cálculos ficarem mais acertiveis, ires fazer apenas com esses dois anos.

* ano de 2023 vai ser o valor de referência - ano anterior que está sendo comparado com o valor atual que no caso iremos usar o de 2024.

* A variação vai ser divida por 2023 porque dividr pela quantidade de transações de 2023 dá uma medida relativa de quanto o valor de 2024 é maior ou menor em relação a 2023. Sem essa divisão, a variação seria apenas uma diferença absoluta, sem contexto de como isso se compara com o total de 2023

* Estados com maior variação positiva indicam um **aumento** substancial nas transações.

* Estados com maior varuação negativa indicam uma **diminuição** substancial nas transações


In [331]:
transacoes_por_estado = dados.groupby(['estado', 'data']).size().reset_index(name='quantidade_transacoes')
transacoes_por_estado['ano'] = transacoes_por_estado['data'].dt.year

total_transacoes_por_estado_ano = transacoes_por_estado.groupby(['estado', 'ano'])['quantidade_transacoes'].sum().reset_index()

# Filtrando os dados para os anos de 2023 e 2024
dados_2023_2024 = total_transacoes_por_estado_ano[total_transacoes_por_estado_ano['ano'].isin([2023, 2024])]

dados_pivotados = dados_2023_2024.pivot(index='estado', columns='ano', values='quantidade_transacoes').reset_index()
dados_pivotados['diferenca_percentual'] = ((dados_pivotados[2024] - dados_pivotados[2023]) / dados_pivotados[2023]) * 100

# Ordenando os estados pela variação percentual
dados_pivotados = dados_pivotados.sort_values('diferenca_percentual', ascending=False)

fig = px.bar(dados_pivotados,
             x='estado',
             y='diferenca_percentual',
             title='Variação Percentual das Transações entre 2023 e 2024 por Estado',
             labels={'diferenca_percentual': 'Variação Percentual (%)', 'estado': 'Estado'},
             color='diferenca_percentual',
             color_continuous_scale='RdYlGn',
             text=dados_pivotados['diferenca_percentual'].round(2),  # Adiciona o texto com a variação
             category_orders={'estado': dados_pivotados['estado'].tolist()})


# Ajustando a posição dos textos
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# Ajustando o layout para melhorar a visualização
fig.update_layout(
    xaxis_title='Estado',
    yaxis_title='Variação Percentual das Transações',
    title_x=0.5,
    plot_bgcolor='white',
    font=dict(size=12),
    xaxis={'categoryorder': 'total descending'}
)
fig.show()

### Notas

### **Ceará teve uma variação percentual superior aos outros estados**

O estado do Ceará teve uma variação positiva e de maior magnitude em relação aos outros estados. Além disso, na página "Sinduscon CE" com o título "Recorde histórico: Mercado Imobiliário de Forteza e Região Metropolitana alcança Valor geral de Vendas de R$ 8.5 bilhoes em 2024"

O mercado imobiliário do Ceará aprensetou um crescimento significativo entre 2023 e 2024. Em 2024, o Valor Geral de Vendas (VGV) em Fortaleza e Região Metropolitana atingiu R$ 8.5 bilhões, representando um aumento de 23% em relação a 2023. Além disso, o volume de financiamentos imobiliários com recursos da poupança alcançou R$ 3.7 bilhões em 2024, o maior dos últimos cinco anos, refletindo um aumento de 84,3% desde 2020.

Fatorias que favorecem esse crescimento expressivo:

* **Demanda por Segunda Moradia**: Houve aumento de 50% na procura por imóveis destinados a segunda residência, especialmente em áreas litorâneas como Meireles e Praia de Iracema.

Fonte: https://investindoporai.com.br/mercado-imobiliario-do-ceara-experimenta-expansao-com-segunda-moradia-e-superpredios

*  **Programas Habitacionais**: Iniciativas como o "Minha Casa, Minha Vida" impulsionaram o setor, facilitando o acesso à moradia e estimulando o mercado

Fonte: https://investindoporai.com.br/mercado-imobiliario-cearense-atinge-numeros-recordes-em-2024-e-projeta-crescimento-para-2025

* **Aumento nos Lançamentos Imobiliários**: Fortaleza registrou um aumento de 233% nos lançamentos imobiliários no primeiro trimestre de 2024 em comparação ao mesmo período do ano anterior, indicando confiança dos incorporadores no mercado

Fonte: https://cbic.org.br/mercado-imobiliario-de-fortaleza-e-regiao-metropolitana-registra-melhor-trimestre-em-9-anos

----------------------------

### **Espirito Santo, Minas Gerais e Parana apresentam queda em transferencias imobiliarias**

A diminuição nas transações imobiliárias nos estados do Espírito Santo (ES), Minas Gerais (MG) e Paraná (PR) pode ser atribuída a diversos fatores econômicos. Foi feito uma pesquisa para buscar causas que poderiam provocar esses acontecimentos

* **Aumento nos Preços dos Imóveis**: Em Vitória, capital do Espírito Santo, os preços dos imóveis registraram um aumento de 12,51% em 2024, atingindo R$ 12.287 por metro quadrado. Esse crescimento é atribuído à escassez de terrenos disponíveis para construção e à busca por qualidade de vida, fatores que podem ter elevado os preços e, consequentemente, reduzido a demanda por imóveis.

Fonte: https://g1.globo.com/es/espirito-santo/noticia/2025/01/08/vitoria-e-a-capital-com-o-metro-quadrado-mais-caro-do-brasil-entenda-os-motivos.ghtml

* **Taxa de Desocupação e Desemprego**: Embora a redução na taxa de desocupação seja geralmente positiva, uma diminuição significativa pode indicar uma escassez de mão de obra disponível, afetando a construção civil e, por consequência, o mercado imobiliário. No **Paraná** e em **Minas Gerais**, houve recuos na taxa de desocupação o que pode ter impactado o setor

Fonte: https://g1.globo.com/es/espirito-santo/noticia/2025/01/08/vitoria-e-a-capital-com-o-metro-quadrado-mais-caro-do-brasil-entenda-os-motivos.ghtml



# 4 - Gere um gráfico de série temporal mostrando a evolução das transações ao longo dos meses

In [332]:
transacoes_por_data = dados.groupby('mes').size().reset_index(name='quantidade_transacoes')

# Calcular a média das transações
media_transacoes = transacoes_por_data['quantidade_transacoes'].mean()

# Criando o gráfico de linha
fig = px.line(transacoes_por_data, x='mes', y='quantidade_transacoes',
              title='Distribuição de Transações por Mês',
              labels={'quantidade_transacoes': 'Número de Transações', 'mes': 'Mês'},
              markers=True)

# Adicionando a linha de média
fig.add_hline(y=media_transacoes, line=dict(color='red', dash='dash'),
              annotation_text=f'Média: {media_transacoes:.2f}',
              annotation_position="bottom right", annotation_font_size=12)

# Formatando o eixo X para mostrar apenas o mês
fig.update_layout(xaxis=dict(tickmode='array', tickvals=transacoes_por_data['mes'], ticktext=[
                    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']),
                  xaxis_title="Mês",
                  yaxis_title="Número de Transações",
                  title_x=0.5,  # Centralizando o título
                  title_font=dict(size=16, family="Arial", color="black"))
fig.show()

A análise aponta para uma tendência de crescimento no mercado imobiliário entre os meses de fevereiro e maio, com um pico de transações notável em maio. Uma possível explicação para esse aumento pode estar relacionada a promoções de início de ano, como ofertas especiais e condições facilitadas que são comuns nesse período. Além disso, durante o mês de fevereiro, há feriados que podem resultar em descontos ou condições atrativas para a compra de imóveis, o que acaba incentivando uma maior atividade nesse intervalo.

No entanto, essa tendência pode mudar à medida que o ano avança. Nos meses finais, especialmente novembro e dezembro, observa-se uma diminuição na procura, uma vez que muitas pessoas se preparam para viagens, festas de fim de ano ou momentos em família, o que acaba desviando a atenção dos negócios imobiliários. Como resultado, muitos deixam decisões de compra ou venda para o ano seguinte, visando um novo planejamento ou retomada das atividades após o período de festas.

Esses padrões de comportamento no mercado podem ser influenciados tanto pela sazonalidade quanto pelos aspectos culturais e sociais, refletindo um comportamento mais comum de compra e venda em momentos específicos do ano.

# 5 - Proponha uma segmentação dos cartórios com base nos critérios que achar relevante.

Utilizaremos o KMEANS para o processo de clusterização para podemos fazer a segmentação dos cartórios

**Clusterização**: é a tarefa de identificar instâncias semelhantes e atribuílas a cluster, ou grupos de instâncias semelhantes. Assim como classificação, cada instância é atribuída a um grupo. No entanto, diferentemente da classificação, a clusterização é uma tarefa não supervisionada.

**Kmeans**: É um algoritmo de aprendizado de máquina do tipo não supervisionado utilizado para clusterização. Ele agrupa os dados em K clusters com base na similaridade entre os pontos, minimizando a soma das distância quadradas entre os pontos e o centro do cluster.

Ele funciona da seguinte maneira:

1 - Escolha do número de cluster (K)

* Definimos quantos clusters deseja formar.

2 - Inicialização dos centroides

* São escolhidos K pontos aleatórios para serem os centroides iniciais dos clusters.

3 - Atribuição dos pontos aos clusters

* Cada ponto do dataset é atribuído ao cluster cujo centroide está mais próximo (com base na distância Euclidiana)

4 - Recalculo dos centroides

* O centroide de cada cluster é atualizado com base na média dos pontos pertencentes a ele.

5 - Repetição até a convergência

* Os passos 3 e 4 são repetidos até que os centroides não mudem mais (ou até atingir um número máximo de iterações)


Segmentação Abordada

Será feito uma segmentação do tamanho do imóvel por tipo do imóvel.

**Identificação de Padrões de Distribuição**: Ao segmentar o tamanho do imóvel por tipo, pode-se identificar padrões ou tendências relacionadas ao tipo de imóvel.


In [333]:
dados_testar = dados.copy()

In [334]:
# Convertendo as variáveis categóricas (estado, cidade) para variáveis numéricas
dados_convertidos = pd.get_dummies(dados_testar, columns= ['estado', 'cidade'])

In [335]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Normalizando os dados
scaler = StandardScaler()
dados_scaled = scaler.fit_transform(dados[['tamanho_imovel']])

In [336]:
dados_cluster = dados.copy()

In [337]:
dados_cluster

,id_cartorio,texto,estado,cidade,data,ano,mes,tamanho_imovel,tipo_transferencia,tipo_imovel,regiao
0,193,Processo Usucapião Extrajudicial Apartamento 1...,PI,Picos,2023-08-01,2023,8,121,Usucapião Extrajudicial,apartamento,Nordeste
1,276,Processo Usucapião Extrajudicial Terreno Rural...,MA,São Luís,2023-11-01,2023,11,35049,Usucapião Extrajudicial,terreno rural,Nordeste
2,70,Processo Usucapião Extrajudicial Terreno Rural...,SC,Blumenau,2024-02-01,2024,2,28034,Usucapião Extrajudicial,terreno rural,Sul
3,277,Casa 291 m² transferido Ana Beatriz Rocha 8910...,CE,Fortaleza,2024-04-01,2024,4,291,Usucapião Extrajudicial,casa,Nordeste
4,166,Processo Compra Venda Casa 153 m² passou Ramos...,RJ,Niterói,2024-10-01,2024,10,153,Compra e Venda,casa,Sudeste
...,...,...,...,...,...,...,...,...,...,...,...
10412,114,Processo Recuperação Crédito Casa 277 m² passo...,PR,Curitiba,2024-10-01,2024,10,277,Recuperação de Crédito,casa,Sul
10413,314,Apartamento 204 m² transferido Ryan Novaes 146...,MG,Uberlândia,2023-03-01,2023,3,204,Usucapião Extrajudicial,apartamento,Sudeste
10414,234,Terreno Rural 43156 m² transferido Mendonça 73...,CE,Sobral,2024-11-01,2024,11,43156,Recuperação de Crédito,terreno rural,Nordeste
10415,72,Apartamento 204 m² transferido Cirino Filhos 7...,SP,Campinas,2024-08-01,2024,8,204,Compra e Venda,apartamento,Sudeste


Pré - processamento de dados categóricos e numéricos

* Iremos fazer a transformação nas variáveis categóricas para transformalas em numéricos, porque nenhum modelo de aprendizado de máquina espera receber dados em formato de texto, iremos aplicar o LabelEncoder para fazer essa transformação.

In [338]:
dados_cluster = dados.copy()
dados_cluster.head(3)

,id_cartorio,texto,estado,cidade,data,ano,mes,tamanho_imovel,tipo_transferencia,tipo_imovel,regiao
0,193,Processo Usucapião Extrajudicial Apartamento 1...,PI,Picos,2023-08-01,2023,8,121,Usucapião Extrajudicial,apartamento,Nordeste
1,276,Processo Usucapião Extrajudicial Terreno Rural...,MA,São Luís,2023-11-01,2023,11,35049,Usucapião Extrajudicial,terreno rural,Nordeste
2,70,Processo Usucapião Extrajudicial Terreno Rural...,SC,Blumenau,2024-02-01,2024,2,28034,Usucapião Extrajudicial,terreno rural,Sul


In [339]:
encoder = LabelEncoder()

dados_cluster['estado'] = encoder.fit_transform(dados['estado'])
dados_cluster['cidade'] = encoder.fit_transform(dados['cidade'])
dados_cluster['tipo_transferencia'] = encoder.fit_transform(dados['tipo_transferencia'])
dados_cluster['tipo_imovel'] = encoder.fit_transform(dados['tipo_imovel'])
dados_cluster['regiao'] = encoder.fit_transform(dados['regiao'])

**Normalização utilizando Logaritmo**

A normalização utilizando o logaritmo é uma técnica usada para transformar dados com distribuições assimétricas ou com grande variabilidade em uma escala mais controlada. Essa transformação é útil quando os dados apresentam valores extremos ou outliers, como é o caso de terrenos rurais, que podem ter valores muito altos.

**Transformação Logarítmica**

Ao aplicar o logaritmo aos dados, você reduz a magnitude das diferenças entre os valores, tornando a distribuição mais próxima de uma distribuição normal. A transformação logarítmica pode ser dada pela seguinte fórmula:

$$
X_{\text{log}} = \log(X)
$$

Onde:
- **X** é o valor original da variável.
- **X_log** é o valor da variável após a transformação logarítmica.

Características da Transformação Logarítmica

1. **Redução da variabilidade**: Ao aplicar o logaritmo, os valores mais altos são comprimidos, reduzindo a variação entre os dados extremos.
2. **Manejo de outliers**: Valores muito altos (outliers) são reduzidos, ajudando a melhorar a análise de dados com grande dispersão.
3. **Distribuição mais próxima da normal**: Muitos algoritmos de machine learning funcionam melhor quando os dados têm uma distribuição mais simétrica, e o logaritmo pode ajudar a atingir isso.

In [340]:
# Normalizando a coluna do tamanho de imoveis
dados_cluster['tamanho_imovel'] = np.log1p((dados_cluster[['tamanho_imovel']]))

In [341]:
# Removendo a coluna de texto
dados_cluster = dados_cluster.drop('texto', axis = 1)

In [342]:
#Removendo a coluna de data
dados_cluster = dados_cluster.drop('data', axis =1)

In [343]:
dados_cluster.head(3)

,id_cartorio,estado,cidade,ano,mes,tamanho_imovel,tipo_transferencia,tipo_imovel,regiao
0,193,16,53,2023,8,4.804021,2,0,1
1,276,9,69,2023,11,10.464531,2,2,1
2,70,23,8,2024,2,10.241209,2,2,4


**Método Elbow**

* Chamada também de método do cotolevo, é uma técnica usada para determinar o número ideal de clusters ao aplicar o algoritmo kmeans ou qualquer outro algoritmo de agrupamento.

* Ele é baseado na análise da inércia do modelo kmeans, ou também chamado de Soma dos Erros Quadráticos Dentro do Cluster, essa inércia é a soma dos quadrados das distâncias entre cada ponto e o centroide de seu respectivo cluster. Ele mede quão bem os dados são agrupados dentro dos clusters. Quanto menor a inércia, melhor a qualidade do agrupamento

* Nesse caso, o melhor ponto foi entre 3 e 4, optaremos por usar o k = 3

In [ ]:
# Pegaremos 2 variáveis para essa segmentacao 
x = dados_cluster[['tamanho_imovel', 'regiao']].copy()

`'k-means++'` é uma técnica de inicialização para o algoritmo K-Means.
- Melhora a escolha dos centróides iniciais, evitando que fiquem muito próximos.
- Ajuda a acelerar a convergência e melhora a qualidade da solução.

**Como funciona:**
1. O primeiro centróide é escolhido aleatoriamente.
2. Os próximos centróides são escolhidos com maior probabilidade de estarem distantes dos anteriores.


Vamos usar ele porque

Escolha inteligente dos centróides: Melhora a inicialização ao escolher centróides mais distantes, evitando soluções ruins.

Melhor convergência: Reduz o número de iterações necessárias para o algoritmo convergir.

Evita resultados inconsistentes: A inicialização aleatória pode gerar diferentes resultados em execuções repetidas, mas o 'k-means++' minimiza esse risco.

In [ ]:
inertia = []
K_range = range(1, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(dados_cluster[['tamanho_imovel', 'tipo_imovel']])
    inertia.append(kmeans.inertia_)

# Criando o gráfico com Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(K_range),
    y=inertia,
    mode='lines+markers',
    name='Inércia',
    marker=dict(size=10, color='blue'),
    line=dict(color='blue', width=2)
))

# Adicionando título e rótulos
fig.update_layout(
    title='Método do Cotovelo',
    xaxis_title='Número de Clusters',
    yaxis_title='Inércia',
    template='plotly_white'  # Altere para 'plotly_white' ou remova essa linha
)

# Exibindo o gráfico
fig.show()

Utilizaremos a métrica Silhouette Score para avaliar a qualidade do agrupamento. Ele mede o quão bem cada ponto de dados se encaixa em seu próprio cluster em comparação com outros clusters.

O valor do Silhouette Score varia de -1 a +1:

- +1: O ponto está muito bem agrupado (fortemente associado ao seu próprio cluster e bem distante dos outros clusters).
-  0: O ponto está entre dois clusters e não pertence claramente a nenhum deles.
- -1: O ponto foi mal agrupado (muito mais próximo de outro cluster do que do seu próprio).

In [ ]:

kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42)
dados_cluster['Cluster'] = kmeans.fit_predict(dados_cluster[['tamanho_imovel', 'tipo_imovel']])

# Verificando o Silhouette Score
score = silhouette_score(dados_cluster[['tamanho_imovel', 'tipo_imovel']], dados_cluster['Cluster'])
print(f"Silhouette Score: {score}")

Silhouette Score: 0.7390076503419519


In [347]:
fig = px.scatter(dados_cluster, x='tamanho_imovel', y='tipo_imovel', color=dados_cluster['Cluster'].astype(str),
                 title='Segmentação por Tamanho do Imóvel e Tipo de Imóvel',
                 labels={'Cluster': 'Grupo'},
                 hover_data=['tamanho_imovel', 'tipo_imovel'])

fig.show()

In [348]:
dados['cluster'] = kmeans.fit_predict(dados_cluster[['tamanho_imovel', 'tipo_imovel']])

In [349]:
dados.head()

,id_cartorio,texto,estado,cidade,data,ano,mes,tamanho_imovel,tipo_transferencia,tipo_imovel,regiao,cluster
0,193,Processo Usucapião Extrajudicial Apartamento 1...,PI,Picos,2023-08-01,2023,8,121,Usucapião Extrajudicial,apartamento,Nordeste,0
1,276,Processo Usucapião Extrajudicial Terreno Rural...,MA,São Luís,2023-11-01,2023,11,35049,Usucapião Extrajudicial,terreno rural,Nordeste,1
2,70,Processo Usucapião Extrajudicial Terreno Rural...,SC,Blumenau,2024-02-01,2024,2,28034,Usucapião Extrajudicial,terreno rural,Sul,1
3,277,Casa 291 m² transferido Ana Beatriz Rocha 8910...,CE,Fortaleza,2024-04-01,2024,4,291,Usucapião Extrajudicial,casa,Nordeste,0
4,166,Processo Compra Venda Casa 153 m² passou Ramos...,RJ,Niterói,2024-10-01,2024,10,153,Compra e Venda,casa,Sudeste,0


Analisando os resultados do clustering, observa-se que as categorias de clusters 1 e 2 foram completamente dominadas por imóveis do tipo "terreno rural", enquanto o cluster 0 apresenta apenas uma pequena proporção desse tipo de imóvel. Além disso, o cluster 0 é mais heterogêneo, dividindo o espaço entre imóveis de "apartamento" e "casa".

In [350]:
tipo_imovel_cluster = dados.groupby('cluster')['tipo_imovel'].value_counts(normalize=True).unstack()

# Criar um gráfico de barras empilhadas para visualizar
fig = px.bar(tipo_imovel_cluster,
             title="Distribuição dos Tipos de Imóvel por Cluster",
             labels={'value': 'Proporção', 'Cluster': 'Cluster'},
             barmode='stack')

fig.show()

In [351]:
# Agrupar os dados por 'cluster' e 'regiao' e contar as ocorrências
cluster_por_regiao = dados.groupby(['cluster', 'regiao']).size().reset_index(name='count')
cluster_por_regiao = cluster_por_regiao.sort_values(by = ['cluster', 'count'], ascending= False)
cluster_por_regiao

,cluster,regiao,count
11,2,Nordeste,287
12,2,Norte,226
13,2,Sudeste,110
10,2,Centro-Oeste,103
14,2,Sul,97
6,1,Nordeste,860
7,1,Norte,759
8,1,Sudeste,360
5,1,Centro-Oeste,320
9,1,Sul,308


- O gráfico de barras mostra que há uma predominância de terrenos rurais (verde) nos clusters 0 e 1, enquanto apartamentos (azul) e casas (vermelho) aparecem majoritariamente no primeiro cluster.

## **Distribuição Regional dos Clusters**

- O **cluster 0** tem uma forte presença nas regiões:
  - **Nordeste** (2194 registros)
  - **Norte** (2015 registros)
  - **Sudeste** (1056 registros)
  - **Centro-Oeste** (881 registros)
  - **Sul** (841 registros)

  Isso sugere que o cluster 0 pode representar uma região específica ou um perfil de imóveis mais comum nessas áreas.

- O **cluster 1** se distribui principalmente entre as regiões:
  - **Nordeste** (860 registros)
  - **Norte** (759 registros)
  - **Sudeste** (360 registros)
  - **Centro-Oeste** (320 registros)
  - **Sul** (308 registros)

  Isso pode indicar um padrão de mercado diferenciado nessas regiões.

- O **cluster 2** é mais frequente nas regiões:
  - **Nordeste** (287 registros)
  - **Norte** (226 registros)
  - **Sudeste** (110 registros)
  - **Centro-Oeste** (103 registros)
  - **Sul** (97 registros)

  Isso pode indicar um segmento de mercado ou faixa de preço específica.



### Analisando cada cluster

- Cluster 0

* Média 174.64 metros quadrados - Imóveis pequenos em geral.
* Mínimo/Máximo: De 5 até 300 metros quadrados, mostrando uma grande variação.
* 1° Quartil (112) e 3° Quartil (237): Mostra que 75% dos imóveis têm no máximo 237 metros quadrados

- Cluster 1

* Média: 31305.21 metros quadrados - Imóveis maiores que os do cluster 0
* Desvio Padrão: 10784.93 - Há uma variação alta nos tamanhos.
* Mínimo/Máximo: De 12753.0 a 49921.0 metros quadrados, ou seja, **não há imóveis pequenos** nesse cluster
* Distribuição: Como a mediana (31322.0) é próxima da média, a distribuiçãa para mais simétrica

- Cluster 2

* Média: 6659.160 metros quadrados
* Desvio Padão: 3356.41	- Possui alta variação.
* Mínimo/Máximo: De 1002 até 12720 metros quadrados
* Distribuição: A mediana (6575.0 metros quadrados)


Cluster 0: Este cluster é predominantemente composto por imóveis de tamanho pequeno a médio, com uma média de 174.64 metros quadrados. A variação de tamanho é significativa, indo de 5 a 300 metros quadrados, mas 75% dos imóveis têm no máximo 237 metros quadrados, o que sugere uma concentração de imóveis menores. Esse cluster parece englobar imóveis comuns em várias regiões, com uma forte presença no Nordeste e Norte.

Cluster 1: Imóveis neste cluster têm um tamanho médio muito maior, com uma média de 31,305 metros quadrados. A faixa de tamanho vai de 12,753 a 49,921 metros quadrados, indicando que são imóveis grandes ou terrenos rurais. A distribuição desses imóveis é mais concentrada nas regiões Nordeste e Norte, e o perfil sugere que são imóveis de alto valor ou específicos de determinadas localizações ou segmentos de mercado.

Cluster 2: Este cluster inclui imóveis de tamanho médio, com uma média de 6,659 metros quadrados. A variação de tamanho vai de 1,002 a 12,720 metros quadrados, com uma mediana de 6,575 metros quadrados, indicando que a distribuição é mais equilibrada. Esse cluster parece representar imóveis com um perfil intermediário, tanto em termos de tamanho quanto de valor, e está mais concentrado em regiões menores como o Sudeste e o Centro-Oeste.

In [352]:
dados.groupby("cluster")["tamanho_imovel"].describe()

,count,mean,std,min,25%,50%,75%,max
cluster,,,,,,,,
0,6987.0,174.646629,72.652797,5.0,112.0,175.0,237.0,300.0
1,2607.0,31305.314921,10784.931929,12753.0,21835.0,31322.0,40666.0,49921.0
2,823.0,6659.160389,3356.412731,1002.0,3824.0,6575.0,9498.0,12720.0


# 6 - Existe alguma relação entre a quantidade de transações e a região geográfica do Brasil?

In [355]:
transacoes_por_regiao = dados.groupby("regiao").size().reset_index(name = 'transacoes')
transacoes_por_regiao = transacoes_por_regiao.sort_values(by = 'transacoes', ascending= False)
print(transacoes_por_regiao)

         regiao  transacoes
1      Nordeste        3341
2         Norte        3000
3       Sudeste        1526
0  Centro-Oeste        1304
4           Sul        1246


In [356]:
# Gráfico de barras
fig = px.bar(transacoes_por_regiao, x="regiao", y="transacoes",
             title="Quantidade de Transações por Região", text_auto=True)
fig.show()


# Criando o gráfico de linhas
transacoes_por_mes_regiao = dados.groupby([dados['data'], 'regiao']).size().reset_index(name='transacoes')
fig = px.line(transacoes_por_mes_regiao,
              x='data',
              y='transacoes',
              color='regiao',
              title="Tendência de Transações por Região ao Longo do Tempo",
              labels={'data': 'Data', 'transacoes': 'Quantidade de Transações'},
              markers=True)

# Exibindo o gráfico
fig.show()

### Notas:

**Nordeste lidera nas transações imobiliarias**
Os dois gráficos mostram que a região do Nordeste possui uma quantidade de transações maiores que os das outras regiões. De acordo com o site "SBR empreendimentos" - na sua matéria com o título "O mercado mobiliário das capitais nodestinas está em plena ascensão" - O mercado imobiliário das capitais nordestinas continua a demonstrar sua força e resiliência, registrando números expressivos no primeiro trimestre de 2024. A região movimentou mais de R$ 6,6 bilhões, refletindo um crescimento de 6% no Valor Geral de Vendas (VGV) em comparação com o mesmo período do ano anterior. Esse desempenho robusto é um indicativo claro de que o setor está em franca expansão, atraindo cada vez mais investidores e compradores em busca de oportunidades sólidas e rentáveis. Investidores estão de olho na valorização contínua dos imóveis, impulsionada tanto pela demanda local quanto pelo interesse de pessoas de outras regiões do Brasil e do exterior. As capitais nordestinas oferecem um mix atrativo de oportunidades, que vão desde empreendimentos residenciais de alto padrão até opções mais acessíveis, como as oferecidas pelo programa Minha Casa, Minha Vida."

Link da matéria: https://sbrempreendimentos.com.br/o-mercado-imobiliario-das-capitais-nordestinas-esta-em-plena-ascensao/

fonte: Ademi-BA

Outra fonte interessante, dessa vez do site  "Movimento Econômico" - 15/02/2025 na sua matéria com o título "Mercado de imóveis de luxo cresce 85% em dois anos no Nordeste"
O mercado de imóveis de luxo no Nordeste cresceu 85% nos últimos dois anos, destacando-se como um setor em ascensão. Em 2024, foram lançadas 1.143 unidades no segmento de luxo e superluxo, representando 3% das vendas da região, mas correspondendo a 25% do valor total das transações imobiliárias. Os dados são da Brain Inteligência Estratégica.

Link da matéria: https://movimentoeconomico.com.br/estados/2025/02/15/mercado-de-imoveis-de-luxo-cresce-85-em-dois-anos-no-nordeste/

fonte: BRAIN | ELABORAÇÂO: BRAIN



**Sudeste - Centro Oeste - Sul com um pouco mais que a metade das transações do Norte e Nordeste**

É notório perceber que essas três regiões apresentaram baixas transações imobiliarias, de acordo com o portal "imoveis.estadao.com.br"  19/09/2023 com a matéria de "Sudeste registra queda em lançamentos e vendas de imóveis no último trimestre" - "Esoecialistas aponta para efeitos da alta taxa de juros, nível de desemprego e turbulência política" "O mercado imobiliário do Sudeste não vive um momento tão positivo em 2023. De acordo com um levantamento da Brain Inteligência Estratégica, o setor registrou queda no número de lançamentos (-18%), vendas (-1,5%) e oferta final (-5,1%) na comparação do último trimestre deste ano com o mesmo período do ano passado. "Quando se analisa o primeiro semestre deste ano e o primeiro semestre de 2022, a queda permanece considerável. A redução no número de lançamentos foi de 16,7%, do número de vendas foi de 2,4% e a queda na oferta final também foi de 5,1%. “A redução de lançamentos justifica-se em especial por um primeiro trimestre fraco”, explica Fábio Tadeu Araújo, CEO da Brain Inteligência Estratégica.

“Isso aconteceu por dois motivos: a economia, impulsionada pela elevada taxa de juros, nível de desemprego e um nível de renda baixo; e a incerteza política, simbolizada pela mudança de governo”, justifica. Na visão dele, o primeiro trimestre do ano é marcado pela indecisão de empreendedores e incorporadores e o adiamento dos lançamentos para o segundo trimestre ou para o final do trimestre."

link da matéria: https://imoveis.estadao.com.br/compra/sudeste-registra-queda-em-lancamentos-e-vendas-de-imoveis-no-ultimo-trimestre/

fonte: Levantamento Mercado Imobiliário da Região Sudeste/ Brain Inteligência Estratégica


### Considerações Finais

Este projeto foi um grande desafio desde o início. Comecei lidando com a extração de dados de uma coluna de texto, algo que exigiu paciência e atenção aos detalhes. Além disso, fui capaz de entender um pouco mais sobre o funcionamento do mercado imobiliário, algo que não conhecia profundamente antes.

Outro ponto desafiador foi a coluna referente ao "tamanho do imóvel", que apresentava uma grande variação nos valores. Esses dados não eram necessariamente outliers, mas a falta de normalização acabou tornando a análise mais complexa. A pesquisa que fiz sobre o setor me abriu os olhos para a realidade do mercado, especialmente no nordeste, onde pude aprender sobre o crescimento das imobiliárias e a valorização do setor.

Ao longo deste processo, me dediquei ao máximo e dei o melhor de mim. Foi uma experiência muito enriquecedora, tanto do ponto de vista técnico quanto do conhecimento do mercado. Estou muito grato pela oportunidade de mostrar minhas habilidades, aprender mais sobre esse ramo e também conhecer mais sobre a empresa e suas operações.

Espero que o meu trabalho tenha sido do seu agrado e que tenha agregado valor. Agradeço pela oportunidade de participar deste projeto.